In [1]:
import xarray as xr
import numpy as np
import datetime
from datetime import date, timedelta
from scipy.stats import bootstrap

In [2]:
import sys
sys.path.insert(0, '/home/cstan/project-git/MJO-Teleconnections/Utils')
from obs_utils import *
from fcst_utils import *
from t2m_utils import *

In [ ]:
print(xr.__version__)

In [3]:
fil_rmm_erai='/projects/cstan/erai/rmm/rmm_ERA-Interim.nc'

In [4]:
ds_rmm=xr.open_dataset(fil_rmm_erai,decode_times=False)

In [5]:
times=ds_rmm['amplitude'].time
init_time=date(1960,1,1)+timedelta(int(times[0]))
time=[]
for i in range(len(times)):
        time.append(init_time+timedelta(i))

In [6]:
import pandas as pd
ds_rmm['time'] = pd.to_datetime(time,format="%Y/%m/%d")

ERA-Interim data covers 01/01/1979-08/31/2019, 7 years and 8 months, 14853 days

In [7]:
fil_t2m_erai='/projects/cstan/erai/t2m/erai.T2m.day.mean.1979-2019.nc'
ds_t2m_erai=xr.open_dataset(fil_t2m_erai)

* Rename lon,lat to match the forecast - useful for plotting

In [8]:
ds_t2m_erai=ds_t2m_erai.rename({'lon': 'longitude','lat': 'latitude'})

Calculate anomalies of observations for the period of forecast 01/01/2011-21/31/2018

In [9]:
tBegin='01-01-2011'
tEnd='12-31-2018'
t2m_obs_anom=calcAnomObs(ds_t2m_erai['t2m'].sel(time=slice(tBegin,tEnd)),'t2m_anom')

Read in forecast data

In [10]:
fil_t2m_ufs_1='/projects/cstan/ufs6/daily/mean/t2m/t2m_*01.nc'
fil_t2m_ufs_15='/projects/cstan/ufs6/daily/mean/t2m/t2m_*15.nc'

In [11]:
ds_t2m_ufs_1=xr.open_mfdataset(fil_t2m_ufs_1,combine='nested',concat_dim='time',parallel=True)
ds_t2m_ufs_15=xr.open_mfdataset(fil_t2m_ufs_15,combine='nested',concat_dim='time',parallel=True)

HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 494 in H5O__attr_open_by_name(): can't locate attribute: '_QuantizeBitGroomNumberOfSignificantDigits'
    major: Attribute
    minor: Object not found
HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Ao

Select all days in November-December-January-February-March

In [12]:
rmm_obs_ndjfm = ds_rmm['amplitude'].sel(time=is_ndjfm(ds_rmm['time.month']))
pha_obs_ndjfm = ds_rmm['phase'].sel(time=is_ndjfm(ds_rmm['time.month']))

Generate time limits for each initial condition 

In [13]:
nyrs=8
yrStrt=2011
mmStrt=1
initial_days=[1, 15]

dStrt=[]
for dd in initial_days:
    dStrt.append(date(yrStrt,mmStrt,dd))
dLast=[]
for i in range(len(initial_days)):
    dLast.append(dStrt[i]+timedelta(days=nyrs*366))

Select the time period of the forecast 01/01/2011-12/31/2018

In [14]:
rmm_obs_1=rmm_obs_ndjfm.sel(time=slice(dStrt[0],dLast[0]))
rmm_obs_15=rmm_obs_ndjfm.sel(time=slice(dStrt[1],dLast[1]))

pha_obs_1=pha_obs_ndjfm.sel(time=slice(dStrt[0],dLast[0]))
pha_obs_15=pha_obs_ndjfm.sel(time=slice(dStrt[1],dLast[1]))

Select initial conditions in the forecast

In [15]:
rmm_fcst_1 = rmm_obs_1.sel(time=is_day1(rmm_obs_1['time.day']))
rmm_fcst_15 = rmm_obs_15.sel(time=is_day15(rmm_obs_15['time.day']))

pha_fcst_1 = pha_obs_1.sel(time=is_day1(pha_obs_1['time.day']))
pha_fcst_15 = pha_obs_15.sel(time=is_day15(pha_obs_15['time.day']))

Select MJO events for MJO phase 3 and 7

In [16]:
phase3 = 3
mjo_events_1_p3 = select_mjo_event(rmm_fcst_1,pha_fcst_1,phase3)
mjo_events_15_p3 = select_mjo_event(rmm_fcst_15,pha_fcst_15,phase3)

phase7 = 7
mjo_events_1_p7 = select_mjo_event(rmm_fcst_1,pha_fcst_1,phase7)
mjo_events_15_p7 = select_mjo_event(rmm_fcst_15,pha_fcst_15,phase7)


Calculate phase composites of **observations** for a given week

In [17]:
week='week4'

var_name='t2m_anom_p3'
obs_comp_anoms_1_p3=calcComposites(t2m_obs_anom,mjo_events_1_p3,week,var_name)
obs_comp_anoms_15_p3=calcComposites(t2m_obs_anom,mjo_events_15_p3,week,var_name)

var_name='t2m_anom_p7'
obs_comp_anoms_1_p7=calcComposites(t2m_obs_anom,mjo_events_1_p7,week,var_name)
obs_comp_anoms_15_p7=calcComposites(t2m_obs_anom,mjo_events_15_p7,week,var_name)

In [18]:
obs_comp_anoms_p3=xr.concat([obs_comp_anoms_1_p3,obs_comp_anoms_15_p3],dim='mjo_events')
obs_comp_anoms_p7=xr.concat([obs_comp_anoms_1_p7,obs_comp_anoms_15_p7],dim='mjo_events')

Calculate statistical significance of composites (**observations**) over the MJO events

In [19]:
n_samples=1000
sig_level=0.95
obs_low_p3,obs_high_p3=test_sig(obs_comp_anoms_p3,sig_level,n_samples)
obs_low_p7,obs_high_p7=test_sig(obs_comp_anoms_p7,sig_level,n_samples)

In [20]:
obs_sig_p3=xr.where((obs_low_p3<0) & (obs_high_p3>0),np.nan,1)
obs_sig_p7=xr.where((obs_low_p7<0) & (obs_high_p7>0),np.nan,1)

Calculate **forecast** anomalies

In [21]:
t2m_anom_ufs_1=calcAnom(ds_t2m_ufs_1['t2m'],'t2m_anom')
t2m_anom_ufs_15=calcAnom(ds_t2m_ufs_15['t2m'],'t2m_anom')

Calculate phase composites of **forecasts** for a given week

In [22]:
week='week4'

var_name='t2m_anom_p3'
ufs_comp_anoms_1_p3=calcComposites(t2m_anom_ufs_1,mjo_events_1_p3,week,var_name)
ufs_comp_anoms_15_p3=calcComposites(t2m_anom_ufs_15,mjo_events_15_p3,week,var_name)

var_name='t2m_anom_p7'
ufs_comp_anoms_1_p7=calcComposites(t2m_anom_ufs_1,mjo_events_1_p7,week,var_name)
ufs_comp_anoms_15_p7=calcComposites(t2m_anom_ufs_15,mjo_events_15_p7,week,var_name)

Combine all MJO events in **forecast**

In [ ]:
ufs_comp_anoms_p3=xr.concat([ufs_comp_anoms_1_p3,ufs_comp_anoms_15_p3],dim='mjo_events')
ufs_comp_anoms_p7=xr.concat([ufs_comp_anoms_1_p7,ufs_comp_anoms_15_p7],dim='mjo_events')

Calculate statistical significance of composites (**forecast**) over the MJO events

In [ ]:
n_samples=1000
sig_level=0.95
ufs_low_p3,ufs_high_p3=test_sig(ufs_comp_anoms_p3,sig_level,n_samples)
ufs_low_p7,ufs_high_p7=test_sig(ufs_comp_anoms_p7,sig_level,n_samples)

In [ ]:
ufs_sig_p3=xr.where((ufs_low_p3<0) & (ufs_high_p3>0),np.nan,1)
ufs_sig_p7=xr.where((ufs_low_p7<0) & (ufs_high_p7>0),np.nan,1)

Plot composites 

In [ ]:
lon_0 = 270
lat_0 = 20
cmap='bwr'
clevs=[-5.0, -4.0, -3.0, -2.0, -1.0, -0.5, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0]

plotComposites(obs_comp_anoms_p3['t2m_anom_p3'].mean(dim='mjo_events'),
               clevs,cmap,lon_0,lat_0,obs_sig_p3)

In [ ]:
plotComposites(obs_comp_anoms_p7['t2m_anom_p7'].mean(dim='mjo_events'),
               clevs,cmap,lon_0,lat_0,obs_sig_p7)

In [ ]:
plotComposites(ufs_comp_anoms_p3['t2m_anom_p3'].mean(dim='mjo_events'),
               clevs,cmap,lon_0,lat_0,ufs_sig_p3)

In [ ]:
plotComposites(ufs_comp_anoms_p7['t2m_anom_p7'].mean(dim='mjo_events'),
               clevs,cmap,lon_0,lat_0,ufs_sig_p7)